## README

### Random forest

#### rilut

##### 100% testing, 100% validation:

```
[('quantity', 0.0079976101188701763),
 ('voucherAmount', 0.013113168566589808),
 ('productGroup', 0.013521791424342219),
 ('paymentMethod', 0.023095387514901027),
 ('voucherID', 0.023673399468102159),
 ('deviceID', 0.02972387355282638),
 ('rrp', 0.034301681510350089),
 ('price', 0.050416333839450621),
 ('sizeCode', 0.081037284731151293),
 ('articleID', 0.099016018175452983),
 ('colorCode', 0.102658103196932),
 ('mmdd', 0.11712529498716262),
 ('orderDate', 0.11765229923719016),
 ('budget', 0.13307758983005333),
 ('total_order', 0.15359016384662513)]
```

error: **40909**

```
[[1106669   10130       4       2       0       0]
 [  30395 1173596      14       0       0       0]
 [     98     105    3546       0       0       0]
 [      5       5       1     181       0       0]
 [      1       2       0       0      32       0]
 [      3       1       0       0       0      24]]
```

##### + fitur farah

```
[('quantity', 0.0017582530808943407),
 ('gpriceDiff', 0.0048983941549550864),
 ('season', 0.0080367194371475381),
 ('voucherAmount', 0.010988671668931234),
 ('productGroup', 0.012766406326011092),
 ('priceDiff', 0.013412936473421951),
 ('voucherID', 0.018625808815826907),
 ('months', 0.019264281253228634),
 ('paymentMethod', 0.021643050043331915),
 ('deviceID', 0.022986859827486265),
 ('sizeCode2', 0.02654573612543476),
 ('price', 0.02953846862187191),
 ('rrp', 0.033643960735023258),
 ('sizeCode', 0.055122931892368984),
 ('orderDate', 0.071741609769720494),
 ('mmdd', 0.072979107307649255),
 ('articleID', 0.080428383834663872),
 ('orderID', 0.086541639359148576),
 ('colorCode', 0.088359528852262254),
 ('customerID', 0.098783688143916887),
 ('budget', 0.1023089129220222),
 ('total_order', 0.11962465135468255)]
```

***error: 40364***

```
[[1075188    9968      14       0       0       0]
 [  29953 1174028      15       0       0       1]
 [    100     142    3507       0       0       0]
 [      4       6       0     182       0       0]
 [      2       4       0       0      29       0]
 [      2       0       0       0       0      26]]
```

In [1]:
label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from datetime import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from __future__ import division
%matplotlib inline
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [2]:
# Dari Farah:

def getSeason(month):
    if month <3 or month==12:
        return "Winter"
    elif month >=3 and month<6:
        return "Spring"
    elif month >=6 and month <9:
        return "Summer"
    else:
        return "Fall"
    
def changeSizeCode(size):
    if size in ['24','25','26','27','28','29','30','31','100']:
        return 'XL'
    elif size in ['32','33','34']:
        return 'XS'
    elif size in ['36','38','75']:
        return 'S'
    elif size in ['40','42','80']:
        return 'M'
    elif size in ['44','85','90','95']:
        return 'L'    
    
def grouppriceDiff(priceDiff):
    if priceDiff < 0:
        return 1
    elif priceDiff == 0:
        return 2
    elif priceDiff > 0 and priceDiff <= 9.99:
        return 3
    else:
        return 4

In [5]:
#df=pd.read_csv('orders_train.txt', sep=';'); df

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity
0,a1000001,2014-01-01,i1000382,1972,44,3,1,10.00,29.99,0,0,c1010575,2,BPRG,0
1,a1000001,2014-01-01,i1000550,3854,44,3,1,20.00,39.99,0,0,c1010575,2,BPRG,0
2,a1000002,2014-01-01,i1001991,2974,38,8,1,35.00,49.99,0,0,c1045905,4,BPRG,0
3,a1000002,2014-01-01,i1001999,1992,38,8,1,49.99,49.99,0,0,c1045905,4,BPRG,1
4,a1000003,2014-01-01,i1001942,1968,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
5,a1000003,2014-01-01,i1001942,1972,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
6,a1000003,2014-01-01,i1001974,1001,42,8,1,25.00,39.99,0,0,c1089295,2,PAYPALVC,0
7,a1000003,2014-01-01,i1001976,3976,44,8,1,15.00,39.99,0,0,c1089295,2,PAYPALVC,0
8,a1000003,2014-01-01,i1002392,2493,42,13,0,0.00,59.99,0,0,c1089295,2,PAYPALVC,0
9,a1000004,2014-01-01,i1002457,1001,42,14,1,89.99,89.99,0,0,c1050116,3,BPRG,1


In [3]:
#joblib.dump(df, 'df.pkl') 
orig_df=joblib.load('df.pkl')

In [4]:
df=orig_df.copy()

In [12]:
df['mmdd']=orig_df.orderDate.str[-5:]
df['months']=orig_df.orderDate.str[-5:-3].astype(np.int)
df['sizeCode2']=df.sizeCode.apply(changeSizeCode)
df['season']= df.months.apply(getSeason)
df['priceDiff'] = df['rrp']-(df['price']/df['quantity'])
df['gpriceDiff'] = df['priceDiff'].apply(grouppriceDiff)

In [13]:
# Berapa total harga dari suatu order

order=df[['orderID', 'price']].groupby('orderID').sum()
order.columns=['total_order']
df=df.set_index('orderID').join(order).reset_index()
del order

In [14]:
# budget rata-rata customer

budget=df[['customerID','total_order']].groupby('customerID').mean()
budget.columns=['budget']
df=df.set_index('customerID').join(budget).reset_index()
del budget

In [15]:
# Return probability. belum selesai
"""
# data training
df_return_probability=df[['articleID','returnQuantity','quantity']].groupby('articleID').sum()
df_return_probability['return_probability']=df_return_probability.returnQuantity/df_return_probability.quantity
del df_return_probability['returnQuantity']
del df_return_probability['quantity']
df_return_probability.columns=['return_probability']
df_return_probability.head()
df.set_index('articleID').join(df_return_probability).reset_index()

# data testing
def get_return_probability(articleID)
    if articleID in df_return_probability.articleID:
        return df_return_probability[articleID].return_probability
    else:
        return 0.5
    
"""

"\n# data training\ndf_return_probability=df[['articleID','returnQuantity','quantity']].groupby('articleID').sum()\ndf_return_probability['return_probability']=df_return_probability.returnQuantity/df_return_probability.quantity\ndel df_return_probability['returnQuantity']\ndel df_return_probability['quantity']\ndf_return_probability.columns=['return_probability']\ndf_return_probability.head()\ndf.set_index('articleID').join(df_return_probability).reset_index()\n\n# data testing\ndef get_return_probability(articleID)\n    if articleID in df_return_probability.articleID:\n        return df_return_probability[articleID].return_probability\n    else:\n        return 0.5\n    \n"

In [16]:
df.head()

,customerID,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,...,paymentMethod,returnQuantity,mmdd,months,sizeCode2,season,priceDiff,gpriceDiff,total_order,budget
0,c1000001,a1001465,2014-01-03,i1003077,3999,I,17,1,27.99,27.99,...,BPRG,0,01-03,1,None,Winter,0,2,27.99,69.618226
1,c1000001,a1004977,2014-01-07,i1003082,1999,I,17,1,25.99,25.99,...,BPRG,1,01-07,1,None,Winter,0,2,25.99,69.618226
2,c1000001,a1012517,2014-01-15,i1001229,1993,38,4,1,39.99,39.99,...,BPRG,0,01-15,1,S,Winter,0,2,39.99,69.618226
3,c1000001,a1013536,2014-01-16,i1002531,1995,38,15,1,59.99,59.99,...,BPRG,1,01-16,1,S,Winter,0,2,119.98,69.618226
4,c1000001,a1013536,2014-01-16,i1002531,1999,40,15,1,59.99,59.99,...,BPRG,1,01-16,1,M,Winter,0,2,119.98,69.618226


In [17]:
#iseng ubah float ke integer
df.price=df.price.astype(np.int)
df.budget=df.budget.astype(np.int)

#Konversi kategori/object ke numerik
print "Konversi kategori/object ke numerik:"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns
for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
#df['price_delta']=df.rrp-df.price;# df.price_delta.head()

Konversi kategori/object ke numerik:
customerID
orderID
orderDate
articleID
sizeCode
voucherID
paymentMethod
mmdd
sizeCode2
season


In [18]:
df.head()

,customerID,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,...,paymentMethod,returnQuantity,mmdd,months,sizeCode2,season,priceDiff,gpriceDiff,total_order,budget
0,0,1452,2,2943,3999,23,17,1,27,27.99,...,2,0,2,1,0,3,0,2,27.99,69
1,0,4937,6,2948,1999,23,17,1,25,25.99,...,2,1,6,1,0,3,0,2,25.99,69
2,0,12400,14,1177,1993,13,4,1,39,39.99,...,2,0,14,1,3,3,0,2,39.99,69
3,0,13403,15,2418,1995,13,15,1,59,59.99,...,2,1,15,1,3,3,0,2,119.98,69
4,0,13403,15,2418,1999,14,15,1,59,59.99,...,2,1,15,1,2,3,0,2,119.98,69


In [19]:
# Selangkah sebelum ditrain

# Drop NaN
df2=df.dropna()

Xdf=df2.drop(label, axis=1)
Xdf.rrp=Xdf.rrp.astype(np.int)

X=Xdf.values
y=df2.returnQuantity.values

In [20]:
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
from itertools import izip as zip
pair=zip(Xdf.columns, rfc.feature_importances_)
t1=[i for i in pair]; t1
#
from operator import itemgetter
sorted(t1, key=itemgetter(1))

[('quantity', 0.0017582530808943407),
 ('gpriceDiff', 0.0048983941549550864),
 ('season', 0.0080367194371475381),
 ('voucherAmount', 0.010988671668931234),
 ('productGroup', 0.012766406326011092),
 ('priceDiff', 0.013412936473421951),
 ('voucherID', 0.018625808815826907),
 ('months', 0.019264281253228634),
 ('paymentMethod', 0.021643050043331915),
 ('deviceID', 0.022986859827486265),
 ('sizeCode2', 0.02654573612543476),
 ('price', 0.02953846862187191),
 ('rrp', 0.033643960735023258),
 ('sizeCode', 0.055122931892368984),
 ('orderDate', 0.071741609769720494),
 ('mmdd', 0.072979107307649255),
 ('articleID', 0.080428383834663872),
 ('orderID', 0.086541639359148576),
 ('colorCode', 0.088359528852262254),
 ('customerID', 0.098783688143916887),
 ('budget', 0.1023089129220222),
 ('total_order', 0.11962465135468255)]

In [22]:
y_pred=rfc.predict(X)
dmc_error(df2.returnQuantity.values, np.array(y_pred))

40364

In [23]:
print confusion_matrix(df2.returnQuantity.values, np.array(y_pred))

[[1075188    9968      14       0       0       0]
 [  29953 1174028      15       0       0       1]
 [    100     142    3507       0       0       0]
 [      4       6       0     182       0       0]
 [      2       4       0       0      29       0]
 [      2       0       0       0       0      26]]


In [25]:
_ = """
for train, test in KFold(len(y), n_folds):
    model.fit(X,y)
    y=model.predict(X)
    return
"""